In [ ]:
def model_based_decoder(options, fmel, fblood, shading, specmask, weightA, weightD, Fweights, CCT, b, BGrid):
        mu = options['mu']
        muim = options['muim']
        PC = options['PC']
        wavelength = options['wavelength']
        illumANorm = options['illumANorm']
        illumDNorm = options['illumDNorm'] 
        illumFNorm = options['illumFNorm']
        T = options['Tmatrix']
        lossweights = options['lossweights']
        newSkinColor = options['NewSkinColor']


        nbatch = b.shape[1]
        #illumination model
        illuminationA = illumANorm * weightA
        illuminationA = illuminationA.to(device) #.to(device) .to('cpu')
        illuminationD = illumDFunc(CCT,illumDNorm)
        #illuminationD = illuminationD.to(device)
        illuminationF = illumFNorm.reshape(1,33,12,1)
        #illuminationF = illuminationF.to(device) #.to(device)

        illFTemp = torch.zeros(Fweights.shape[0],33,12,1)
        for i in range(Fweights.shape[0]):
          for j in range(12):
            illFTemp[i,:,j,:,] = illuminationF[:,:,j,:,]*Fweights[i,j,:,:,]
        

        illuminationF = torch.sum(illFTemp,2)
        illuminationF = illuminationF.permute(0, 2, 1)
        illuminationF = illuminationF.to(device)
        e = illuminationA + illuminationD + illuminationF
        a = torch.sum(e,2)
        
        for i in range(nbatch):
          e[i,:,:,] = e[i,:,:,] / a[i]

        
        relS = nn.ReLU()
        #camera Model
        
        
        S = torch.zeros(PC.shape[0],nbatch)
        for j in range(b.shape[1]):
          for i in range(PC.shape[0]):  
            S[i,0] = PC[i,0]*b[0,j] + PC[i,1]*b[1,j] + mu[i,0]
        
        S = relS(S)
        Sr = S[0:wavelength,:,]
        Sg = S[wavelength:2*wavelength]
        Sb = S[2*wavelength:3*wavelength]

        Sr = Sr.reshape(nbatch, 1, wavelength)
        Sr = Sr.to(device)
        Sg = Sg.reshape(nbatch, 1, wavelength)
        Sg = Sg.to(device)
        Sb = Sb.reshape(nbatch, 1, wavelength)
        Sb = Sb.to(device)

        #compute Light Color
        
        lightcolorR = torch.sum(Sr * e,2).to(device)
        lightcolorG = torch.sum(Sg * e,2).to(device)
        lightcolorB = torch.sum(Sb * e,2).to(device)
        
        lightcolor = torch.cat((lightcolorR, lightcolorG, lightcolorB),1).to(device)
        
        lightcolor = lightcolor.reshape(lightcolor.shape[0],1,1, 3)
        
        #compute Specularities
        
        specmask = specmask.reshape(nbatch, specmask.shape[1],specmask.shape[2],1 )
        specularities = specmask*lightcolor
        
        #bio map
        fmel = fmel.reshape(1,fmel.shape[0],fmel.shape[1],fmel.shape[2])
        fblood = fblood.reshape(1,fblood.shape[0],fblood.shape[1],fblood.shape[2])
        biomaps = torch.cat((fmel,fblood),0)
        biomaps = biomaps.permute(1,0,2,3)
        
        newSkinTemp = torch.zeros(nbatch, newSkinColor.shape[0],newSkinColor.shape[1],newSkinColor.shape[2])
        
        for i in range(nbatch):
          newSkinTemp[i,:,:,:,] = newSkinColor

        newSkinColor = newSkinTemp
        newSkinColor = newSkinColor.permute(0,3,1,2)
        #print(newSkinColor.shape, biomaps.shape)

        R_total = nn.functional.interpolate(newSkinColor, size = (biomaps.shape[2],biomaps.shape[3]), mode = 'bilinear')
        R_total = R_total.to(device)
        #print(R_total.shape)        

        for j in range(nbatch):  
          for i in range(e.shape[2]):
            R_total[j,i,:,:,] = R_total[j,i,:,:,] * e[j,:,i]
        
        spectraRef = R_total
        
        rChanTemp = torch.zeros(spectraRef.shape) 
        rChanTemp = rChanTemp.to(device)
        gChanTemp = torch.zeros(spectraRef.shape) 
        gChanTemp = gChanTemp.to(device)
        bChanTemp = torch.zeros(spectraRef.shape) 
        bChanTemp = bChanTemp.to(device)
        for j in range(nbatch):  
          for i in range(e.shape[2]):
            rChanTemp[j,i,:,:,] = spectraRef[j,i,:,:,] * Sr[j,:,i] 
            gChanTemp[j,i,:,:,] = spectraRef[j,i,:,:,] * Sg[j,:,i] 
            bChanTemp[j,i,:,:,] = spectraRef[j,i,:,:,] * Sb[j,:,i] 

        rChannel = torch.sum(rChanTemp,1).to(device)
        gChannel = torch.sum(gChanTemp,1).to(device)  
        bChannel = torch.sum(bChanTemp,1).to(device)  

        rChannel = rChannel.reshape(rChannel.shape[0],1,rChannel.shape[1],rChannel.shape[2])
        gChannel = bChannel.reshape(gChannel.shape[0],1,gChannel.shape[1],gChannel.shape[2])
        bChannel = bChannel.reshape(bChannel.shape[0],1,bChannel.shape[1],bChannel.shape[2])
        

        diffuseAlbedo = torch.cat((rChannel,gChannel,bChannel), 1 ).to(device)
        
        shading = shading.reshape(nbatch,1,shading.shape[1],shading.shape[2])
        specularities = specularities.reshape(nbatch,specularities.shape[3],specularities.shape[1],specularities.shape[2])
        
        shadedDiffuse = diffuseAlbedo

        for j in range(nbatch):  
          for i in range(diffuseAlbedo.shape[1]):
            shadedDiffuse[j,i,:,:,] = diffuseAlbedo[j,i,:,:,] * shading[j,:,:,:,] 
        
        rawAppearance = shadedDiffuse + specularities
        
        WBrCh = torch.zeros(nbatch, 1, rawAppearance.shape[2], rawAppearance.shape[3])
        WBgCh = WBrCh
        WBbCh = WBgCh
        for j in range(nbatch):  
            WBrCh[j,:,:,:,] = rawAppearance[j,0,:,:,]/lightcolor[j,:,:,0,]
            WBgCh[j,:,:,:,] = rawAppearance[j,1,:,:,]/lightcolor[j,:,:,1,]
            WBbCh[j,:,:,:,] = rawAppearance[j,2,:,:,]/lightcolor[j,:,:,2,]

        
        imWhiteBalanced = torch.cat((WBrCh,WBgCh,WBbCh),1)
        imWhiteBalanced = imWhiteBalanced.to(device)
        
        T = T.reshape(1,T.shape[2],T.shape[0],T.shape[1])
        #T = T.permute(2,0,1)
        #Ttemp = torch.zeros(nbatch, T.shape[0],T.shape[1],T.shape[2])
        
        #for i in range(nbatch):
          #Ttemp[i,:,:,:,] = T
        #T = Ttemp
        #print(T.shape,BGrid.shape)
        #print(BGrid)
        Trawxyz = nn.functional.interpolate(T, size = (BGrid.shape[2],BGrid.shape[3]), mode = 'bilinear') #(BGrid.shape[3], BGrid.shape[0])
        Trawxyz = Trawxyz.permute(3,0,2,1)
        #Trawxyz = Trawxyz.to(device)

        #print(Trawxyz.shape)
        Ix = torch.zeros(nbatch, 1,imWhiteBalanced.shape[2],imWhiteBalanced.shape[3])
        Iy = torch.zeros(nbatch, 1,imWhiteBalanced.shape[2],imWhiteBalanced.shape[3])
        Iz = torch.zeros(nbatch, 1,imWhiteBalanced.shape[2],imWhiteBalanced.shape[3])

        #print(imWhiteBalanced.shape)
        for j in range(nbatch):
          Ix[j,:,:,:,] = Trawxyz[j,:,:,0] * imWhiteBalanced[j,0,:,:,] + Trawxyz[j,:,:,3] * imWhiteBalanced[j,1,:,:,] + Trawxyz[j,:,:,6] * imWhiteBalanced[j,2,:,:,]
          Iy[j,:,:,:,] = Trawxyz[j,:,:,1] * imWhiteBalanced[j,0,:,:,] + Trawxyz[j,:,:,4] * imWhiteBalanced[j,1,:,:,] + Trawxyz[j,:,:,7] * imWhiteBalanced[j,2,:,:,]
          Iz[j,:,:,:,] = Trawxyz[j,:,:,2] * imWhiteBalanced[j,0,:,:,] + Trawxyz[j,:,:,5] * imWhiteBalanced[j,1,:,:,] + Trawxyz[j,:,:,8] * imWhiteBalanced[j,2,:,:,]

        Txyzrgb = torch.tensor([[3.2406, -1.5372, -0.4986],
                               [-0.9689, 1.8758, 0.0415],
                               [0.0557, -0.2040, 1.057]]
                               )
        Ixyz = torch.cat((Ix,Iy,Iz),1)
        rgbim = torch.zeros(nbatch,3,Ixyz.shape[2],Ixyz.shape[3])
        #print(rgbim.shape)
        for j in range(nbatch):
          rgbim[j,0,:,:,] = Txyzrgb[0,0] * Ixyz[j,0,:,:,] + Txyzrgb[0,1] * Ixyz[j,1,:,:,] + Txyzrgb[0,2] * Ixyz[j,2,:,:,] #r
          rgbim[j,1,:,:,] = Txyzrgb[1,0] * Ixyz[j,0,:,:,] + Txyzrgb[1,1] * Ixyz[j,1,:,:,] + Txyzrgb[1,2] * Ixyz[j,2,:,:,] #g
          rgbim[j,2,:,:,] = Txyzrgb[2,0] * Ixyz[j,0,:,:,] + Txyzrgb[2,1] * Ixyz[j,1,:,:,] + Txyzrgb[2,2] * Ixyz[j,2,:,:,] #b

        rgbim[:,0,:,:,] = rgbim[:,0,:,:,] * 255 #- Y1[:,:,0]
        rgbim[:,1,:,:,] = rgbim[:,1,:,:,] * 255 #- Y1[:,:,1]
        rgbim[:,2,:,:,] = rgbim[:,2,:,:,] * 255 #- Y1[:,:,2]
        #rgbim = relS(rgbim)
        a = 0.055
        
        #rgbim = (1+a)(rgbim**(12/5)) - a
        
        #final nonlinear gamme transformation
        #gamma = 2.4
        #simulating rgbim^2.4 by doing rgbim^(24/10) below

        rgbim = rgbim**24
        for i in range(rgbim.shape[0]):
          for j in range(rgbim.shape[2]):
            for k in range(rgbim.shape[3]):
              for num_Sqrt in range(5): #
                rgbim[i,0,j,k] = torch.sqrt(rgbim[i,0,j,k]) #(1+a)(rgbim[i,0,j,k,]**(1/5)) - a
                rgbim[i,1,j,k] = torch.sqrt(rgbim[i,1,j,k]) #(1+a)(rgbim[i,1,j,k,]**(1/5)) - a
                rgbim[i,2,j,k] = torch.sqrt(rgbim[i,2,j,k]) #(1+a)(rgbim[i,2,j,k,]**(1/5)) - a
        
        rgbim = (1+a)*rgbim - a


        return rgbim, specularities

In [ ]:
def illumDFunc(CCT,illumDNorm):
  S_0 = torch.zeros(CCT.shape[0],1,33).to(device)
  S_1 = torch.zeros(CCT.shape[0],1,33).to(device)
  S_2 = torch.zeros(CCT.shape[0],1,33).to(device)
  
  for k in range(CCT.shape[0]):
    for i in range(33):
      for j in range(22):
        S_0[k,0,i] = (illumDNorm[0,0,i,j]/22) + S_0[k,0,i]

  illMat = illumDNorm[0,0,:,:,] #.to(device)
  
  #e, v = torch.eig(illMat, eigenvectors = True) #e are eigenvalues & v are eigenvectors
  u, s, v = torch.svd(illMat)
  S_1 = u[:,0].to(device)
  S_2 = u[:,1].to(device)
  x_D = torch.zeros((CCT.shape[0])).to(device)
  y_D = torch.zeros((CCT.shape[0])).to(device)
  M = torch.zeros((CCT.shape[0])).to(device)
  M_1 = torch.zeros((CCT.shape[0])).to(device)
  M_2 = torch.zeros((CCT.shape[0])).to(device)
  illD = torch.zeros(CCT.shape[0],1,33).to(device)

  for i in range(CCT.shape[0]):
    cct = CCT[i,:,:,]
    if cct >= 4000 and cct < 7000: #CCT[i,:,:,] >= 4000 & CCT[i,:,:,] < 7000 : #CCT
      x_D[i] = 0.244063 + 0.09911*(1e3)/CCT[i,:,:,] + 2.9678*(1e6)/(CCT[i,:,:,]*CCT[i,:,:,]) - 4.6070*(1e9)/(CCT[i,:,:,]*CCT[i,:,:,]*CCT[i,:,:,])
      y_D[i] = -3.000*x_D[i]*x_D[i] + 2.870*x_D[i] - 0.275
    else: #CCT[i,:,:,] >= 7000 & CCT[i,:,:,] <= 25000 :
      x_D[i] = 0.237040 + 0.24748*(1e3)/CCT[i,:,:,] + 1.9018*(1e6)/(CCT[i,:,:,]*CCT[i,:,:,]) - 2.0064*(1e9)/(CCT[i,:,:,]*CCT[i,:,:,]*CCT[i,:,:,])
      y_D[i] = -3.000*x_D[i]*x_D[i] + 2.870*x_D[i] - 0.275

    M[i] = 0.0241 + 0.2562*x_D[i] - 0.7341*y_D[i]
    M_1[i] = (-1.3515 - 1.7703*x_D[i] + 5.9114*y_D[i])/M[i]
    M_2[i] = (-0.03000 - 31.4424*x_D[i] + 30.0717*y_D[i])/M[i]
    illD[i,:,:] = S_0[i,:,:,] + M_1[i] * S_1 + M_2[i] * S_2

  illDTemp = torch.zeros(1,1,33).to(device)
  for i in range(CCT.shape[0]):
    illDTemp = illDTemp + illD[i,:,:,]/CCT.shape[0]
  
  illuminationD = illDTemp.to(device)

  return illuminationD